In [ ]:
pip install cassandra-driver

In [55]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    "secure_connect_bundle": "secure-connect-nyc-yellow-taxi-db.zip"  # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider(
    "yiGvbsUpoEqllXBsXXwOjZBc",
    "p,b-6bj-cMT1IR3dv,kGm6A8Q+yZg0jcMHWivO2XU0y9+evSLpYZ923mLvs,ocJ-CLeOdYI8EeiBoa7jR.r3fOFtpiQwO+Zdb2aq4FqaO4xSEXj_TF4KFkHoAs5N6mt_",
)  # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace(
    "nyc_yellow_taxi_db")  # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

In [56]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

# read the csv files into the dataframes
trip_data = pd.read_csv('taxi_trip_data.csv')
zone_geo = pd.read_csv('taxi_zone_geo.csv')

In [57]:
# a) Remove the columns “store_and_fwd_flag”, “rate_code” and “total_amount” from taxitripdata
trip_data = trip_data.drop(columns=['store_and_fwd_flag', 'rate_code', 'total_amount'])

In [68]:
# Assuming df_trips is the DataFrame for taxi trips and df_zones is the DataFrame for geographic zones

# Merge the two DataFrames on location information
trip_data = pd.merge(
    trip_data, zone_geo, left_on="pickup_location_id", right_on="zone_id", how="left"
)

trip_data = pd.merge(
    trip_data, zone_geo, left_on="dropoff_location_id", right_on="zone_id", how="left"
)


In [69]:
# Shuffle the DataFrame and then select the first 25,000 records
trip_data = trip_data.sample(
    frac=1
)  # frac=1 means shuffling the entire DataFrame

In [72]:
# print the first 5 rows of the dataframe
print(trip_data.head())

         vendor_id      pickup_datetime     dropoff_datetime  passenger_count   
240236           1  2018-04-27 18:34:08  2018-04-27 18:36:57                1  \
333314           4  2018-12-08 15:19:02  2018-12-08 15:29:42                1   
4393603          1  2018-02-03 00:55:14  2018-02-03 00:58:03                1   
6376074          2  2018-04-23 14:18:13  2018-04-23 14:23:55                2   
2070947          2  2018-08-26 16:36:02  2018-08-26 16:49:16                1   

         trip_distance  payment_type  fare_amount  extra  mta_tax  tip_amount   
240236            0.40             1          4.0    1.0      0.5        0.65  \
333314            1.20             1          8.5    0.0      0.5        1.86   
4393603           0.50             2          4.0    0.5      0.5        0.00   
6376074           1.29             1          6.5    0.0      0.5        1.46   
2070947           1.96             2         10.5    0.0      0.5        0.00   

         ...  pickup_locat

In [71]:
# b) Drop rows with missing essential details that would be required to fulfill the upcoming queries
trip_data = trip_data.dropna()

In [81]:
# c) Insert the data in the database as you see fit

# create the table for trip_data, with a UDT for the pickup and dropoff location from the zone_geo table
session.execute(
    """
    CREATE TYPE IF NOT EXISTS location (
        zone_id int,
        zone_name text,
        borough text,
        zone_geom text
    )
    """
)

# columns in the trip_data table:
# vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, rate_code, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount,
# imp_surcharge, total_amount, pickup_location_id, dropoff_location_id, pickup_location, dropoff_location
#  zone_id_x zone_name_x  borough_x  zone_geom_x  zone_id_y zone_name_y  borough_y  zone_geom_y

# create the table for trip_data
session.execute(
    """
    CREATE TABLE IF NOT EXISTS trip_data (
        vendor_id text,
        pickup_datetime timestamp,
        dropoff_datetime timestamp,
        passenger_count int,
        trip_distance float,
        payment_type text,
        fare_amount float,
        extra float,
        mta_tax float,
        tip_amount float,
        tolls_amount float,
        imp_surcharge float,
        total_amount float,
        pickup_location frozen<location>,
        dropoff_location frozen<location>,
        PRIMARY KEY (vendor_id, pickup_datetime)
    )
    """
)


# c) Insert the data in the database as you see fit

# create the table for trip_data, with a UDT for the pickup and dropoff location from the zone_geo table
session.execute(
    """
    CREATE TYPE IF NOT EXISTS location (
        zone_id int,
        zone_name text,
        borough text,
        zone_geom text
    )
    """
)

# columns in the trip_data table:
#  vendor_id pickup_datetime dropoff_datetime  passenger_count trip_distance  payment_type  fare_amount  extra  mta_tax  tip_amount tolls_amount  imp_surcharge  pickup_location_id  dropoff_location_id

# columns in the zone_geo dataframe:
# zone_id, zone_name, borough, zone_geom

# create the table for trip_data
session.execute(
    """
    CREATE TABLE IF NOT EXISTS trip_data (
        vendor_id text,
        pickup_datetime timestamp,
        dropoff_datetime timestamp,
        passenger_count int,
        trip_distance float,
        payment_type text,
        fare_amount float,
        extra float,
        mta_tax float,
        tip_amount float,
        tolls_amount float,
        imp_surcharge float,
        pickup_location frozen<location>,
        dropoff_location frozen<location>,
        PRIMARY KEY (vendor_id, pickup_datetime)
    )
    """
)

# insert the data from the trip_data dataframe and from the zone_geo dataframe into the trip_data table

for x in range(0, 25000):
    vendor_id = int(trip_data['vendor_id'][x])
    pickup_datetime = trip_data['pickup_datetime'][x]
    dropoff_datetime = trip_data['dropoff_datetime'][x]
    passenger_count = int(trip_data['passenger_count'][x]) # convert the passenger_count to an int
    trip_distance = trip_data['trip_distance'][x]
    payment_type = trip_data['payment_type'][x]
    fare_amount = trip_data['fare_amount'][x]
    extra = trip_data['extra'][x]
    mta_tax = trip_data['mta_tax'][x]
    tip_amount = trip_data['tip_amount'][x]
    tolls_amount = trip_data['tolls_amount'][x]
    imp_surcharge = trip_data['imp_surcharge'][x]
    pickup_location_id = trip_data['pickup_location_id'][x]
    zone_id_x = int(trip_data['zone_id_x'][x]) # convert the zone_id to an int
    zone_name_x = trip_data['zone_name_x'][x]
    borough_x = trip_data['borough_x'][x]
    zone_geom_x = trip_data['zone_geom_x'][x]
    dropoff_location_id = trip_data['dropoff_location_id'][x]
    zone_id_y = int(trip_data['zone_id_y'][x]) # convert the zone_id to an int
    zone_name_y = trip_data['zone_name_y'][x]
    borough_y = trip_data['borough_y'][x]
    zone_geom_y = trip_data['zone_geom_y'][x]


    # insert the data into the trip_data table
    session.execute(
        """
        INSERT INTO trip_data (vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, pickup_location, dropoff_location)
        VALUES (%d, %s, %s, %d, %f, %s, %f, %f, %f, %f, %f, %f, {zone_id: %d, zone_name: %s, borough: %s, zone_geom: %s}, {zone_id: %d, zone_name: %s, borough: %s, zone_geom: %s})
        """,
        (vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, {'zone_id': zone_id_x, 'zone_name': zone_name_x, 'borough': borough_x, 'zone_geom': zone_geom_x}, {'zone_id': zone_id_y, 'zone_name': zone_name_y, 'borough': borough_y, 'zone_geom': zone_geom_y})
    )

TypeError: %d format: a real number is required, not str

In [ ]:
# d) Calculate the duration for each trip and add it as a new field in your database (e7sb el duration men pickup date le dropoff date)
# add a new column to the trip_data dataframe for the duration of the trip
trip_data["duration"] = trip_data["dropoff_datetime"] - trip_data["pickup_datetime"]

# add a new column to the trip_data table for the duration of the trip
session.execute(
    """
    ALTER TABLE trip_data ADD duration timestamp
    """
)

# update the duration column in the trip_data table with the duration of the trip
for x in range(0, 25000):
    pickup_datetime = trip_data["pickup_datetime"][x]
    dropoff_datetime = trip_data["dropoff_datetime"][x]
    duration = dropoff_datetime - pickup_datetime

    session.execute(
        """
        UPDATE trip_data
        SET duration = %s
        WHERE pickup_datetime = %s AND dropoff_datetime = %s, vendor_id = %s
        """,
        (duration, pickup_datetime, dropoff_datetime, trip_data["vendor_id"][x]),
    )

In [ ]:
# e) Use “fare_amount”, “extra”, “mta_tax”, “tip_amount”, “tolls_amount” and “imp_surcharge” to calculate the total trip cost and add it as a new field in your database (e7sb el total cost)


In [ ]:
# f) What is the most common payment type used per time of day? Hint: time of day meaning morning, afternoon, evening, night
# create a new column in the trip_data dataframe for the time of day
trip_data["time_of_day"] = pd.cut(
    trip_data["pickup_datetime"].dt.hour,
    bins=[0, 6, 12, 18, 24],
    labels=["night", "morning", "afternoon", "evening"],
)

# create a new column in the trip_data table for the time of day
session.execute(
    """
    ALTER TABLE trip_data ADD time_of_day text
    """
)

# update the time_of_day column in the trip_data table with the time of day
for x in range(0, 25000):
    time_of_day = pd.cut(
        trip_data["pickup_datetime"].dt.hour,
        bins=[0, 6, 12, 18, 24],
        labels=["night", "morning", "afternoon", "evening"],
    )

    session.execute(
        """
        UPDATE trip_data
        SET time_of_day = %s
        WHERE pickup_datetime = %s AND dropoff_datetime = %s, vendor_id = %s
        """,
        (time_of_day, trip_data["pickup_datetime"][x], trip_data["dropoff_datetime"][x], trip_data["vendor_id"][x]),
    )

# query the trip_data table to find the most common payment type used per time of day
result = session.execute(
    """
    SELECT time_of_day, payment_type, COUNT(*)
    FROM trip_data
    GROUP BY time_of_day, payment_type
    """
)


In [ ]:
# g) What is the average tip amount per passenger count?
# query the trip_data table to find the average tip amount per passenger count
result = session.execute(
    """
    SELECT passenger_count, AVG(tip_amount)
    FROM trip_data
    GROUP BY passenger_count
    """
)

# print the average tip amount per passenger count
for row in result:
    print(row)

In [ ]:
# h) What are the best 5 locations for drivers to pick up passengers from? (based on the trip count)
# query the trip_data table to find the best 5 locations for drivers to pick up passengers from
result = session.execute(
    """
    SELECT pickup_location.zone_name, COUNT(*)
    FROM trip_data
    WHERE pickup_location.zone_name IS NOT NULL
    GROUP BY pickup_location.zone_name
    ORDER BY COUNT(*) DESC
    LIMIT 5
    """
)

# print the best 5 locations for drivers to pick up passengers from
for row in result:
    print(row)  

In [ ]:
# Is there a correlation between trip distance and the tip amount? (not to be done using the correlation calculation)
# create a new cloumn tip percentage, and calculate the tip percentage for each trip by dividing the tip amount by the total amount, and compare it to the trip distance

In [ ]:
# Display the results of f, g and h using visualizations (will be graded based on the creativity and efficiency of the visualizations)
